In [1]:
from pathlib import Path
import tiffile as tif
import numpy as np
import multiscale.utility_functions as util
import os
import h5py

In [13]:
def save_us_image(path, bmode, spacing):
    print('Saving {}'.format(path))
    ijstyle = bmode
    shape = ijstyle.shape
    ijstyle.shape = 1, shape[0], 1, shape[1], shape[2], 1
    tif.imwrite(path, ijstyle.astype(np.float32), imagej=True,
                resolution=(1./spacing[0], 1./spacing[1]),
                metadata={'spacing': spacing[2], 'unit': 'um'})
    
def open_image(path):
    try:
        mat = util.load_mat(str(path))
        mat['IQData'] = np.transpose(mat['IQData'], [2, 0, 1])
        return util.load_mat(str(path))
    except NotImplementedError:
        mat = {}
        with h5py.File(str(path), 'r') as file:
            iqdata = np.array(file['IQData'])
            vectorized_tocomplex = np.vectorize(to_complex)
            mat['IQData'] = vectorized_tocomplex(iqdata)
            mat['IQData'] = np.transpose(mat['IQData'], [0, 2, 1])
            mat['IQData'] = np.flip(mat['IQData'], 0)
            param = dict(file['param'])
            for key in param:
                param[key] = param[key][0][0]
            mat['param'] = param
        return mat
    
def to_complex(element):
    return element[0] + 1j*element[1]
    
def get_spacing(data_dict):
    shape = np.shape(data_dict['IQData'])
    params = data_dict['param']
    lat_spacing = params['BmodeWidth']/shape[2]
    ax_spacing = params['BmodeDepth']/shape[1]
    spacing = [lat_spacing*1000, ax_spacing*1000, params['StepSize3D']*1000]
    return spacing

def iq_to_bmode(IQData, epsilon=0.001):
    return 20*np.log10(np.abs(IQData)+epsilon)

def convert_to_tif(path, output_path):
    data = open_image(path)
    bmode = iq_to_bmode(data['IQData'])
    spacing = get_spacing(data)
    save_us_image(output_path, bmode, spacing)
    
    

In [3]:
base_path = r'C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19'

In [4]:
name_list = ['Mam003', 'Mam205', 'Mam209', 'Mam211', 'Mam215', 'Mam1204']

output_folder = r'C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19'

In [5]:
transducer_names = ['MS700', 'MS400', 'MS550D']


In [14]:
for name in range(len(transducer_names)):
    acquisition_folder = Path(output_folder, transducer_names[name])
    os.makedirs(str(acquisition_folder), exist_ok=True)
    for image in range(len(name_list)):
        input_path = Path(base_path, transducer_names[name], name_list[image] + '.mat')
        output_path = Path(output_folder, transducer_names[name], name_list[image] + '.tif')
         if output_path.is_file():
            continue
        convert_to_tif(input_path, output_path)

Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19\MS700\Mam003.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19\MS700\Mam205.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19\MS700\Mam209.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19\MS700\Mam211.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19\MS700\Mam215.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19\MS700\Mam1204.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19\MS400\Mam003.tif
Saving C:\Users\mpinkert\Box\Eliceiri transducer testing_2019-05-01-14-07-44_Weichmann A\IQ files 5-15-19\MS400\Mam205.tif
Saving C:\Users

In [6]:
test_path = Path(base_path, transducer_names[0], name_list[0] + '.mat')
data = open_image(test_path)

In [7]:
np.shape(data['IQData'])

(447, 512, 664)